# $k$ Nearest Neighbor
## Foundations of Machine Learning
## `! git clone https://www.github.com/DS3001/knn`

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Algorithms and Models
- The analysis we've done up to now is about cleaning and summarizing the properties of data
- We want to pivot to using the data to make predictions
- In machine learning, there are two key concepts:
  - *Algorithms*: A process that converts inputs into results
  - *Models*: A mathematical abstraction or simplification intended to mimic real phenomena
- **Machine learning**: *We (1) use algorithms to fit models to data, and then (2) use those fitted models to make predictions about future outcomes for new data*
- The goal is to produce a data-driven tool for predictive purposes
- There is a lot to learn, and the steps will become more natural as we fit and use more models

## Two Algorithmic Steps
- There's two steps: (1) Fitting the model and (2) Making the predictions
- We'll introduce the model by working backwards: Given a fitted model, how do you make predictions? Understanding how models work gives you insights into how to build them.
- With an understanding of how the model works, we'll then take a step back: How do you train/fit/tune/etc. the model? This is a *training algorithm* or *optimization algorithm*.
- In common practice, the difference between machine learning and statistics is that ML is focused on *prediction* and less on *inference* or *causality* --- the goal is to build predictive models, not estimate them and engage in *hypothesis testing* (unbiased/consistent estimates are not necessarily [often not] good predictive models)

## Data Examples
- Car Price (`USA_cars_datasets.csv`): Based on car attributes ($X$), predict its transaction price ($y\ge 0$)
- Heart Failure (`heart_failure_clinical_records_dataset.csv`): Based on patient characteristics ($X$), predict whether they die ($y=0,1$)

## Data in General
- We have a clean $N \times (M+1)$ matrix $D$ of data; rows $i$ are observations and columns $j$ are variables
- One variable is be the *target*, *dependent*, *outcome*, *prediction*, *labeled* variable, which is an $N \times 1$ vector $y$
- All the other variables are potential *feature*, *explanatory*, *independent*, *control*, *covariate*, *predictor* variables, which is an $N \times M$ matrix $X$
- We'll refer to the $i$-th row/observation of $X$ by $x_i$

## Euclidean Distance
- Distance is at the core of many of our models (kNN, kMC, kernel methods), so let's review the basic defintions before we start with kNN
- Take two numbers, like 5 and 7. What's the distance between them? You can start by subtracting them, but the order then seems to matter: $5-7=-2$ and $7-5=2$. So we square that difference, and then take the square root again to convert to the absolute value:
$$
d(z,w) = \sqrt{(z-w)^2} = |z-w|
$$
    - That gives us $\sqrt{(5-7)^2} = \sqrt{(-2)^2} = \sqrt{4}=2$.

## Distance in Multiple Dimensions
- Distance on the real line between two numbers is intuitive, but what about in multiple dimensions?
- Take `z = (3,5,7)` and `w=(2,-4,6)`
- Then the squared differences are $(3-2)^2=1$, $(5--4)^2=81$, and $(7-6)^2=1$
- The sum is 83
- The square root is 9.110
```
np.sqrt((3-2)**2+(5--4)**2+(7-6)**2)
```

## Euclidean Distance
- What about the distance between two vectors $z = (z_1, z_2, ..., z_L)$ and $w = (w_1, w_2, ..., w_L)$? For example, two rows of a dataframe?
- To measure the *(Euclidean) distance* between two vectors, we do "the same thing":
    1. For each dimension $1, 2, ..., L$, compute the squared difference, $(z_\ell-w_\ell)^2$
    2. Sum the squared differences, $S =(z_1-w_1)^2+ ... + (z_L-w_L)^2$
    3. Take the square root of the sum, $\sqrt{S}$
- More compactly:
$$
d(z,w) = \sqrt{\underbrace{(z_1-w_1)^2}_{\text{First Dimension}} + ... + \underbrace{(z_L-w_L)^2}_{\text{$L$-th Dimension}}} =  \sqrt{\sum_{\ell=1}^L (z_{\ell} - w_{\ell})^2} = ||z-w||
$$
Again, distance is such a ubiquitous idea that there are many notations for it.

## Other Kinds of Distance
- There are many ways to think of the distance between two objects, and they all have strengths and weaknesses:
    - Sup/Max norm: $ d_\infty(z,w) = \max_{i} |z_i-w_i|$ is useful for worst-case situations
    - Euclidean norm: $ d_2(z,w) = \sqrt{\sum_{\ell=1}^L (z_{\ell} - w_{\ell})^2}$, the standard distance metric
    - Manhattan norm: $d_1(z,w) = \sum_{\ell=1}^L |z_\ell - w_\ell|$ is more robust to outliers than Euclidean
    - $p$-norm: $ d_p(z,q) = \left( \sum_{\ell=1}^L |z_\ell-w_\ell|^p \right)^{1/p}$ generalizes all the previous cases ($p=2$ for Euclidean, $p=\infty$ for max, $p=1$ for Manhattan)
- It often depends on context: How do you measure the social distance between people?
- This is extremely important for many algorithms.

## k Nearest Neighbor
- The following claim seems reasonable: "Suppose that a target variable $y$ follows from features $x$. If $x$ and $x'$ are close together, then $y$ and $y'$ are probably also close together." **On some level, this is the core premise of machine learning.**
- If that's true, then making a prediction $\hat{y}$ for a new case $\hat{x}$ could be done by finding the values of $x$ in the data close to $\hat{x}$, and averaging those values
- We'll operationalize this idea more precisely in the subsequent slides

## Car Data
- The data are from the Environmental Protection Agency, and contain some interesting variables:
    - Manufacturer: Honda, BMW, etc.
    - EPA class: SUV, Midsize Car, etc.
    - Footprint: A measure of carbon emissions averaged over time
    - baseline price: Market price of the vehicle
    - baseline mpg: manufacturer's claim of miles per gallon
    - baseline sales: Predicted sales of vehicle
- We want to start by doing some EDA

In [6]:
df = pd.read_csv('./data/cars_env.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/cars_env.csv'

In [ ]:
df.describe()

In [ ]:
sns.kdeplot(data=df,x='baseline price')

In [ ]:
# Drop extremely expensive cars:
q90 = np.quantile( df['baseline price'],.9) # Compute the .9 quantile
# drop 90 % of data
print(q90)
keep = df['baseline price'] < q90  # Logical condition asserting price < .9 quantile
df = df.loc[keep,:] # Use locator function to filter on a Boolean conditional
df.describe()
sns.kdeplot(data=df,x='baseline price')

In [ ]:
df['EPA class'].value_counts() # This is too fine-grained a set of distinctions for our purposes

In [ ]:
# Simplify the vehicle classification scheme:
# aggregate values (group them together)
df['class'] = df['EPA class']
df['class'] = df['class'].replace(['MIDSIZE CARS','COMPACT CARS','SUBCOMPACT CARS','TWO SEATERS','LARGE CARS'],'car')
df['class'] = df['class'].replace(['SMALL STATION WAGONS','MIDSIZE STATION WAGONS'],'station wagon')
df['class'] = df['class'].replace(['STANDARD PICKUP TRUCKS','SMALL PICKUP TRUCKS'],'truck')
df['class'] = df['class'].replace(['VANS','MINIVAN'],'van')

In [ ]:
df['class'].value_counts()

In [ ]:
df.loc[:,['baseline price','EPA class']].groupby('EPA class').describe() # Baseline price by EPA vehicle class

In [ ]:
df.loc[:,['baseline price','class']].groupby('class').describe() # Baseline price by simplified vehicle class

In [ ]:
# Plot footprint against price
this_plot = sns.scatterplot(data=df,x='footprint',y='baseline price',
                            hue='baseline sales',style='class',
                           palette = 'crest')
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

In [7]:
# Plot sales against price
this_plot = sns.scatterplot(data=df,x='baseline price',y='baseline sales',hue='footprint',style='class',
                           palette = 'crest')
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

NameError: name 'df' is not defined

In [ ]:
df.loc[:,['footprint','class']].groupby('class').describe() # Footprint by simplified vehicle class

## Prediction: Regression and Classification
- Let's focus on the graph below: a scatterplot of `baseline price` and `footprint`, with sales coded by hue and vehicle class coded by the marker
- Let's stipulate that we're interested in two questions:
    - Conditional on price and footprint, what are sales likely to be? (numeric outcome)
    - Conditional on price and footprint, what class of vehicle is it likely to be? (categorical outcome)
- Using data to predit a numeric outcome is called **regression**, and using data to predict a categorical outcome is called **classification**
- We often denote the **target variable** or **outcome variable** by $y$ and a prediction by $\hat{y}$, and we denote the **features** or **covariates** by $x$
- The two approaches -- regression and classification -- have some substantial differences that we'll discuss as we go along

In [ ]:
# Plot footprint against price
this_plot = sns.scatterplot(data=df,x='footprint',y='baseline price',
                            hue='baseline sales',style='class',
                           palette = 'crest')
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## $k$ Nearest Neighbor Prediction Algorithm (Regression)
- Consider a new case $\hat{x} = (\hat{x}_1,...,\hat{x}_L)$. We want to make a guess of what value it will likely take, $\hat{y}$, given existing data $D = (X,y)$
- Suppose the outcome variable $y$ is numeric: Price, weight, time, decibels, etc.
- The *$k$ Nearest Neighbor Regression Algorithm* is:
  1. Compute the distance from $\hat{x}$ to each observation $x_i$
  2. Find the $k$ "nearest neighbors" $x_1^*$, $x_2^*$, ..., $x_K^*$ to $\hat{x}$ in the data, with outcomes $y_{1}^*$, $y_2^*$, ..., $y_K^*$
  3. Compute the average nearest neighhor outcome as the prediction for $\hat{x}$:
  -the modal case among the nieghbors (cheater bc 52% chance used chatgpt)
  - a probability of each class occurring, proportional to the frequency in the nieghbors  (52% used chat but also 48% didn't use it)

$$
  \hat{y} = \dfrac{y_1^* + y_2^* + ... + y_K^* }{K} =  \dfrac{1}{K} \sum_{k=1}^K y_k^*
$$


In [ ]:
def knn_reg(x_hat,gdf,K):
    # Compute distances between x_hat and the data:
    squared_differences = (x_hat - gdf.loc[:,['x1','x2']])**2
    distances = np.sum( squared_differences , axis = 1)
    # Find k smallest values in dist:
    neighbors = np.argsort(distances)[:K].tolist()
    # Find y values for the nearest neighbors:
    y_star = gdf['y'].iloc[neighbors].tolist()
    # Average neighbor values to get prediction:
    y_hat = np.mean(y_star)
    # Return a dictionary of computed values of interest:
    return({'y_hat':y_hat, 'y_star':y_star, 'neighbors':neighbors})

In [ ]:
# Cleaning the data a bit; we'll come back to this later
gdf = df.loc[:,['footprint', 'baseline price',  'baseline sales', 'class',]]
gdf = gdf.rename(columns={'footprint':'x1', 'baseline price':'x2', 'class':'g','baseline sales':'y'})

def maxmin(z):
    w = (z-np.min(z))/(np.max(z)-np.min(z))
    return w

gdf['x1'] = maxmin(gdf['x1'])
gdf['x2'] = maxmin(gdf['x2'])
gdf['y'] = maxmin(gdf['y'])

gdf.head()

In [ ]:
# Example:
x_hat = np.array((.2,.2)) # Example prediction case
knn = knn_reg(x_hat, gdf, 3) # Call our function
print(knn)

# Plot results:
gdf2 = gdf.copy() # Notice the .copy(); why?
hat_row = [ x_hat[0], x_hat[1],knn['y_hat'],'hat']
gdf2.loc[len(gdf2)] = hat_row
this_plot = sns.scatterplot(data=gdf2,x='x1',y='x2',hue='y',style='g',palette='crest') # Plot the fake data
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## $k$ Nearest Neighbor Prediction Algorithm (Classification)
- Consider a new case $\hat{x} = (\hat{x}_1,...,\hat{x}_L)$. We want to make a guess of what **class** it will likely take, $\hat{g}$, given existing data $D = (X,g)$
- Suppose the outcome variable $y$ is categorical: Model, species, country, etc.
- The *$k$ Nearest Neighbor Classification Algorithm* is:
  1. Compute the distance from $\hat{x}$ to each observation $x_i$
  2. Find the $k$ "nearest neighbors" $x_1^*$, $x_2^*$, ..., $x_K^*$ to $\hat{x}$ in the data, with outcomes $g_{1}^*$, $g_2^*$, ..., $g_K^*$
  3. Compute either
        - The modal case among the neighbors, $g_{1}^*$, $g_2^*$, ..., $g_K^*$
        - A probability of each class occuring, proportional to the frequency in the neighbors $g_{1}^*$, $g_2^*$, ..., $g_K^*$

In [ ]:
# A simple implementaton of kNN for classification:
def knn_class(x_hat,gdf,K):
    # Compute distances between x_hat and the data:
    squared_differences = (x_hat - gdf.loc[:,['x1','x2']])**2
    distances = np.sum( squared_differences , axis = 1)
    # Find k smallest values in dist:
    neighbors = np.argsort(distances)[:K].tolist()
    # Find g values for the nearest neighbors:
    g_star = gdf['g'].iloc[neighbors]
    # Modal class:
    g_dist = g_star.value_counts()/K
    g_modal = g_dist.index[g_dist.argmax()]
    # Return a dictionary of computed values of interest:
    return({'neighbors':neighbors, 'g_star':g_star, 'g_dist':g_dist, 'g_modal':g_modal, })

In [ ]:
# Example:
x_hat = np.array((.8,.2)) # Example prediction case
knn = knn_class(x_hat, gdf, 3) # Call our function
print(knn)

gdf2 = gdf.copy()
hat_row = [ x_hat[0], x_hat[1],100, knn['g_modal']]
print(hat_row)
gdf2.loc[len(gdf2)] = hat_row
this_plot = sns.scatterplot(data=gdf2,x='x1',y='x2',hue='y',style='g', palette='crest') # Plot the fake data
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## SciKit-Learn
- Unless we are doing something tailored to a particular task, we generally don't want to code our own algorithms: It is time consuming, and existing implementations are typically more efficient or robust than what we would create (but there is a lot of value in coding your own algorithms and estimators)
- The most popular Python machine learning library is called **SciKit-Learn**
- You typically import it as `from sklearn.<model class> import <model name>`, where `<model class>` is the set of related models and `<model name>` is the desired algorithm
    - In particular, `from sklearn.neighbors import KNeighborsRegressor` and `from sklearn.neighbors import KNeighborsClassifier`
- The workflow with `sk` is that you use it to
  1. Create an untrained model object with a fixed $k$: `model = kNNRegression(n_neighbors=k)`
  2. Fit that object to the data, $(X,y)$: `fitted_model = model.fit(X,y)`
  3. Use the fitted object to make predictions for new cases $\hat{x}$: `y_hat = fitted_model.predict(x_hat)`

## Fitting a Model with kNN
- To prepare to fit the model we have to do a few things:
    - Import the model we want to use: `from sklearn.neighbors import KNeighborsRegressor`
    - Create a matrix `X` of features and a vector `y` for the target variable
    - **Scale the features in `X` so their values are comparable**
- Once we have our desired model imported and the data prepared as $D=(y,X)$, we can fit the model

## Feature/Variable Normalization/Scaling
- Problem: the distances in $k$NN depends on the "size/scale" of variables. Multiplying one variable by 10 or dividing by 100 (but not the others) will likely change your predictions
- Solution: We often **scale** or **normalize** variables to reduce the extent to which their relative values change the performance of the model
- There are a variety of approaches to this, but the simplest is **MaxMin Normalization**: For a vector $x$ and element $x_i$,
$$
u_i = \dfrac{x_i - \min(x)}{\max(x)-\min(x)}
$$
- You do not have to normalize the outcome variable, $y$, since we don't look at any distances related to $y$ in fitting or using the model (in some algorithms, you would want to)

## The `.apply()` Method
- We'll make a maxmin scaler function:
```
def maxmin(z):
    z = (z-min(z))/(max(z)-min(z))
    return(z)
```
- But then we want to use it on each column of the data `X`
- Instead of a `for` loop, we can instead use the `df.apply(fcn)` method to apply the function `fcn` to each column

In [ ]:
from sklearn.neighbors import KNeighborsRegressor # Import the kNN regression tool

In [ ]:
# Select features/target variable for analysis:
y = df['baseline sales'] # Create target variable vector
X = df.loc[:,['baseline price','footprint']] # Create feature matrix
print(X.head())
print(X.describe())

In [ ]:
# Maxmin normalization function:
def maxmin(z):
    z = (z-min(z))/(max(z)-min(z))
    return(z)

# Apply maxmin to each column of X to get U:
X = X.apply(maxmin)
print(X.head())
print(X.describe())

In [ ]:
# Fit the model:
k=3
model = KNeighborsRegressor(n_neighbors=k) # Create a sk model for k=3
fitted_model = model.fit(X,y) # Train the model on our data

## Prediction
- Now that we have a fitted model, we can make predictions with it
- You can pass dataframes of $\hat{x}$ values into the `.predict(x_hat)` method of a fitted model
- This returns a set of predictions $\hat{y}$
- Let's plot the model predictions on a grid to see what it does
- The next code chunk is just a tool to make a picture, you don't need to follow each line

In [ ]:
N_x = 100 # Coarseness of x variable
N_y = 100 # Coarseness of y variable
total = N_x*N_y # Total number of points to plot

grid_x = np.linspace(0,1,N_x) # Create a grid of x values
grid_y = np.linspace(0,1,N_y) # Create a grid of y values

xs, ys = np.meshgrid(grid_x,grid_y) # Explode grids to all possible pairs
X = xs.reshape(total) # Turns pairs into vectors
Y = ys.reshape(total) # Turns pairs into vectors

x_hat = pd.DataFrame({'baseline price':X,'footprint':Y}) # Create a dataframe of points to plot
y_hat = fitted_model.predict(x_hat) # Fit the model to the points
x_hat['Predicted Sales'] = y_hat # Add new variable to the dataframe
# Create seaborn plot:
this_plot = sns.scatterplot(data=x_hat,x='baseline price',y='footprint',
                            hue='Predicted Sales', palette = 'crest', linewidth=0)
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## Changing the number of neighbors
- We'll look at how to pick $k$ in a few slides
- Before we do that, how does the predictive model change as $k$ changes?
- Basically, the decision boundaries will get "softer" and the predictions will become less binary
- At some point, the model turns to mush: Instead of using local information, you're using "most of the data", and the predictions will tend towards the sample average
- It is often useful to push models to extremes to see how they "break"
- **Underfitting** occurs when your model is too simple to reliably explain the phenomenon you are interested in, and **overfitting** occurs when your model is too complex to reliably explain the phenomenon you are interested in
- Notice the minimum and maximum predicted values in the legend, and the coarseness of the decision boundaries

In [ ]:
def statics(k=3,data=df): # This function replicates our previous work, with k as an input
    y = df['baseline sales'] # Create target variable vector
    X = df.loc[:,['baseline price','footprint']] # Create feature matrix
    # Normalize:
    X = X.apply(maxmin)
    model = KNeighborsRegressor(n_neighbors=k) # Create a sk model for k=3
    fitted_model = model.fit(X,y) # Train the model on our data
    N_x = 100 # Coarseness of x variable
    N_y = 100 # Coarseness of y variable
    total = N_x*N_y # Total number of points to plot
    grid_x = np.linspace(0,1,N_x) # Create a grid of x values
    grid_y = np.linspace(0,1,N_y) # Create a grid of y values
    xs, ys = np.meshgrid(grid_x,grid_y) # Explode grids to all possible pairs
    X = xs.reshape(total) # Turns pairs into vectors
    Y = ys.reshape(total) # Turns pairs into vectors
    x_hat = pd.DataFrame({'baseline price':X,'footprint':Y}) # Create a dataframe of points to plot
    y_hat = fitted_model.predict(x_hat) # Fit the model to the points
    x_hat['Predicted Sales'] = y_hat # Add new variable to the dataframe
    # Create seaborn plot:
    this_plot = sns.scatterplot(data=x_hat,x='baseline price',y='footprint',
                                hue='Predicted Sales',palette='crest',linewidth=0)
    sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

In [ ]:
statics(2)

In [ ]:
statics(3)

In [ ]:
statics(4)

In [ ]:
statics(5)

In [ ]:
statics(10)

In [ ]:
statics(20)

In [ ]:
statics(50)

In [ ]:
statics(100)

In [ ]:
statics(200)

In [ ]:
statics(470)

## Comparative Statics on $k$
- As $k$ goes up, the prediction regions get fuzzier
- The range of predicted values shrinks, because you're averaging over more values and extremes get moderated
- Eventually, you're averaging over so many neighbors that you make roughly the same predictions for everyone
- So the model can be unrealistically precise ($k$ is very small) or unrealistically imprecise ($k$ is very large)

## What happens if we fail to scale?
- Well, it doesn't "work", it looks really bad: This is an explanation of what not to do
- Since `baseline price` is in tens of thousands and `footprint` is in tens, we get "vertical" equivalence classes of predictions: The distance between one variable are uniformly larger than distances between the other, so there's immense pressure to pick neighbors with similar prices rather than footprints
- But we want to use both dimensions somewhat equally.
- When using algorithms based on distance metrics, you really have to make sure that the way you conceive of "distance" across dimensions makes sense.
- (An alternative is using smoothing to weight neighbors, like a kernel density plot, rather than a hard boundary that picks specific neighbors, like a histogram.)

In [ ]:
# Select features/target variable for analysis:
y = df['baseline sales'] # Create target variable vector
X = df.loc[:,['baseline price','footprint']] # Create feature matrix
model = KNeighborsRegressor(n_neighbors=3) # Create a sk model for k=3
fitted_model = model.fit(X,y) # Train the model on our data

# Make a plot of the decision rule:
N_x = 100 # Coarseness of x variable
N_y = 100 # Coarseness of y variable
total = N_x*N_y # Total number of points to plot
xmin = np.min(X.iloc[:,0])
xmax = np.max(X.iloc[:,0])
ymin = np.min(X.iloc[:,1])
ymax = np.max(X.iloc[:,1])
grid_x = np.linspace(xmin,xmax,N_x) # Create a grid of x values
grid_y = np.linspace(ymin,ymax,N_y) # Create a grid of y values
xs, ys = np.meshgrid(grid_x,grid_y) # Explode grids to all possible pairs
X = xs.reshape(total) # Turns pairs into vectors
Y = ys.reshape(total) # Turns pairs into vectors
x_hat = pd.DataFrame({'baseline price':X,'footprint':Y}) # Create a dataframe of points to plot
y_hat = fitted_model.predict(x_hat) # Fit the model to the points
x_hat['Predicted Sales'] = y_hat # Add new variable to the dataframe
# Create seaborn plot:
this_plot = sns.scatterplot(data=x_hat,x='baseline price',y='footprint',
                            hue='Predicted Sales',palette='crest',linewidth=0)
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

In [ ]:
this_plot = sns.scatterplot(data=df,x='baseline price',y='footprint',hue='baseline sales',
                           palette = 'crest')
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## What is a good fit?
- In machine learning, one of the main concerns is **external validity** or **generalizability**: Will the model perform well in practice, on data that it wasn't trained to fit?
- A **heuristic/rule-of-thumb** is $k^* = \lfloor \sqrt{N} \rfloor$. For the car purchase data, $k^*=21$. But this isn't really data-driven at all.
- There are roughly two concerns: Models will overutilize features of the data that might be specific to this data and not be present in the population overall, and model **hyper parameters** like $k$ will tend to **overfit** and focus on feature of the data that are unlikely to appear in a new sample
- In general, models that fit well on training data are not guaranteed in any way to fit well on new data (when predictive performance matters)

## Core Ideas of Model Selection
- In regression, we start with the **errors** or **residuals** for each observation:

    \begin{alignat*}{1}
    e_i = y_i - \hat{y}_i(x_i,k)
    \end{alignat*}

    This quantifies how far off the $i$-th prediction is from the actual value
- In classification, we start with the confusion matrix that cross tabulates how close our predictions were to the true values. In cases where $0$ is negative and $1$ is positive, we have:

|           |          |                | Actual         |
|-----------|----------|----------------|----------------|
|           |          | Negative       | Positive       |
| Predicted | Negative | True Negative  | False Negative |
|           | Positive | False Positive | True Positive  |

More generally, we can cross tabulate the predicted with actual values for any number of categories (like vehicle class)

## Fit and Sum of Squared Error
- What is a good fit?
- We usually have a distance metric in mind: "How far are the predictions from the true outcomes, on the validation/test set?"
- So for each observation $i$ in the validation set, we subtract the predicted value from the true one and square it, then sum over all the observations:
$$
SSE(k) = \sum_{i=1}^L (y_i - \hat{y}(x_i,k))^2 = \sum_{i=1}^L e_i^2
$$
- The $SSE(k)$ is a really common measure of how far the predicted values are from the true values

## Fitting the Model: Selecting $k$

- A simple and effective data-driven strategy is to randomly **split the data** into two sets, a **training set** and a **test/validation set**:
  - For a reasonable range of values $k = 1, 2, ..., \overline{K}$, we make a prediction $\hat{y}_i$ using the training set for each observation in the test set, $x_i$
  - Compute the squared difference from the prediction $\hat{y}_i$ to the true outcome $y_i$ for each point $i$ in the test set, then sum to get the *sum of squared error (SSE)*
  - Pick the $k$ that achieves the lowest SSE
- The variable $k$ is our first example of a *hyperparameter*: The data themselves do not prescribe a value of $k$, and naive ways of picking it will lead to bad predictive models

## Splitting the Sample
- `sk` has a `.model_selection` module that contains a `train_test_split` function that makes it easy to split your data
- Once the data are split, we use a `for` loop to compute the SSE for the training and test data for each reasonable value of $k$

In [ ]:
y = df['baseline sales'] # Create target variable vector
X = df.loc[:,['baseline price','footprint']] # Create feature matrix
X = X.apply(maxmin) # Normalize X

# Split the sample:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, # Feature and target variables
                                                    test_size=.2, # Split the sample 80 train/ 20 test
                                                    random_state=100) # For replication purposes

In [ ]:
k_bar = 200 # Number of k's to try
SSE = np.zeros(k_bar) # We'll store the SSE here
SSE_on_train = np.zeros(k_bar) # For pedogogical purposes, we'll save the training error

for k in range(k_bar):
    model = KNeighborsRegressor(n_neighbors=k+1) # Create a sk model for k
    fitted_model = model.fit(X_train,y_train) # Train the model on our data
    y_hat = fitted_model.predict(X_test) # Predict values for test set
    SSE[k] = np.sum( (y_test-y_hat)**2 ) # Save the computed SSE for test set
    y_hat = fitted_model.predict(X_train) # Predict values for training set
    SSE_on_train[k] = np.sum( (y_train-y_hat)**2 ) # Save the computed SSE

SSE_min = np.min(SSE) # Find lowest recorded SSE
min_index = np.where(SSE==SSE_min) # Find the indices of SSE that equal the minimum
k_star = min_index[0]+1 # Find the optimal value of k; why index+1?
print(k_star)

In [ ]:
plt.plot(np.arange(0,k_bar),SSE,label='Test SSE') # Plot SSE by k
plt.xlabel("k")
plt.ylabel("SSE")
plt.legend(loc='upper right')
plt.title('SSE')
plt.show()

In [ ]:
plt.plot(np.arange(0,k_bar),SSE_on_train,label='Train') # Plot SSE by k
plt.plot(np.arange(0,k_bar),SSE,label='Test') # Plot SSE by k
plt.xlabel("k")
plt.ylabel("SSE")
plt.legend(loc='upper left')
plt.title('SSE')
plt.show()

## Training Error vs. Test Error
- Why does the training error increase in $k$?
- Or, what is the training SSE for $k=1$?

## Additional Variables
- The only reason we only used two variables is so we could visualize how the algorithm was working
- You can use as many variables as you want, but in high dimensional spaces, everything is far apart and model performance breaks down

## Regression versus Classification
- Our first application was a *regression* problem: Predicting a numeric or continuous value, which was sales
- Our next application is *classification*: Predicting which values are most likely when the outcome is categorical, which is vehicle category
- For example, instead of predicting sales, the goal might be to predict the class or make/model based on characteristics like price, engine size, fuel economy, number of doors, etc.

## Classification Example
- To illustrate how classification works, we are going to predict class from baseline price and footprint
- With classification, the goal is to either the probability that the new case will belong to each category or predict the most likely category

In [ ]:
this_plot = sns.scatterplot(data=df,x='baseline price',y='footprint',hue='class')
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Select features/target variable for analysis:
y = df['class']
X = df.loc[:,['baseline price','footprint']] # Create feature matrix

# Max-min normalize the features:
X = X.apply(maxmin)

# Fit the model:
model = KNeighborsClassifier(n_neighbors=3) # Create a sk model for k=3
classifier_model = model.fit(X,y) # Train the model on our data

In [ ]:
# Graph of predictor:
N_x = 100 # Coarseness of x variable
N_y = 100 # Coarseness of y variable
total = N_x*N_y # Total number of points to plot
grid_x = np.linspace(0,1,N_x) # Create a grid of x values
grid_y = np.linspace(0,1,N_y) # Create a grid of y values
xs, ys = np.meshgrid(grid_x,grid_y) # Explode grids to all possible pairs
X = xs.reshape(total) # Turns pairs into vectors
Y = ys.reshape(total) # Turns pairs into vectors
x_hat = pd.DataFrame({'baseline price':X,'footprint':Y}) # Create a dataframe of points to plot
y_hat = classifier_model.predict(x_hat) # Fit the model to the points
x_hat['Predicted Class'] = y_hat # Add new variable to the dataframe
# Create seaborn plot:
this_plot = sns.scatterplot(data=x_hat,x='baseline price',y='footprint',hue='Predicted Class', linewidth=0)
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

## Confusion Matrices
- The simplest criteria of success with a classifier is the proportion of cases correctly predicted: The **Accuracy**
- To more broadly measure the performance of a classifier, we can cross-tabulate the predicted and true values in a **Confusion Matrix**, and look for patterns in the successes and failures
- We'll revisit these ideas later in more depth: Accuracy is not a popular criterion for model success
- Like test error, we typically need to splt the sample and compute this on hold-out data, not the training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

## Select data:
y = df['class'] # Target variable
X = df.loc[:,['baseline price','footprint']] # Create feature matrix
X = X.apply(maxmin) # Normalize X

In [ ]:
## Split the sample:
X_train, X_test, y_train, y_test = train_test_split(X,y, # Feature and target variables
                                                    test_size=.5, # Split the sample 80 train/ 20 test
                                                    random_state=200) # For replication purposes
N_train = len(y_train)
N_test = len(y_test)

In [ ]:
## Solve for k that maximizes accuracy:
k_bar = 100 # Number of k's to try
Acc = np.zeros(k_bar) # We'll store the accuracy here

for k in range(k_bar):
    model = KNeighborsClassifier(n_neighbors=k+1) # Create a sk model for k
    fitted_model = model.fit(X_train.values,y_train) # Train the model on our data
    y_hat = fitted_model.predict(X_test.values) # Predict values for test set
    Acc[k] = np.sum( y_hat == y_test )/N_test # Accuracy on testing data

Acc_max = np.max(Acc) # Find highest recorded Accuracy
max_index = np.where(Acc==Acc_max) # Find the indices that equal the maximum
k_star = max_index[0]+1 # Find the optimal value of k; why index+1?
print(k_star)

In [ ]:
## Fit optimal model:
model = KNeighborsClassifier(n_neighbors=k_star[0]) # Create a sk model for k
fitted_model = model.fit(X_train.values,y_train) # Train the model on our data
y_hat = fitted_model.predict(X_test.values) # Predict values for test set

In [ ]:
## Accuracy plot:
plt.plot(np.arange(0,k_bar),Acc,label='Accuracy')
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.legend(loc='upper right')
plt.title('Test Accuracy')
plt.show()

In [ ]:
## Confusion matrix:
pd.crosstab(y_test,y_hat)

- Notice how, because of the random train/test split, no station wagons or vans made it into the training data, so the algorithm only predicts SUV, car, and truck
- Despite that, it predicts that station wagons are most likely to be cars and vans are most likely to be trucks: It still does an OK job of guessing what the vehicle is closest to

## Other Examples
- heart_failure.csv: `DEATH_EVENT` is that the patient died, and all of the other varibles are potential predictors
- airbnb_hw.csv: Price as a function of characteristics

## Issues with $k$-NN
- This is a purely inductive/blackbox prediction algorithm: We find the $k$ cases that are "most similar" to $\hat{x}$ in terms of distance, and guess the average of the outcomes for those points
- The choice of $d$ is crucial but apparently arbitrary
- For regression, it returns a *point prediction*, $\hat{y}$, and we might want to allow for more uncertainty in our predictions
- It is unclear how to interpret the impact of individual variables on predictions because of the local smoothing
- The local smoothing is "hard" like a histogram and not "smooth" like a kernel density; this leads to instability in predictions
- $k$NN is our "chunkiest", "most local" predictive algorithm, so it is often good for imputation

## Main Take-aways
There is a lot in this lecture, if you are new to predictive modeling. We will revist most of it in future classes, but here are some key ideas:
- Euclidean Distance
- Regression versus Classification
- Feature Scaling/Normalization
- Splitting the Sample and Hyperparameter Fitting
- Sum of Squared Error
- kNN regression and classification
- The `.apply(fcn)` method